In [1]:
import pika

In [2]:
#connection = pika.BlockingConnection(
#    pika.ConnectionParameters(host='localhost'))
#channel = connection.channel()

In [3]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

In [4]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

In [5]:
def on_request(ch, method, props, body):
    n = int(body)

    print(" [.] fib(%s)" % n)
    response = fib(n)

    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)


In [6]:
channel.queue_declare(queue='rpc_queue')
channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='rpc_queue', on_message_callback=on_request)

'ctag1.5787d9da669c43ffbe4c60ad2380960c'

In [ ]:
print(" [x] Awaiting RPC requests")
channel.start_consuming()

 [x] Awaiting RPC requests
 [.] fib(30)
 [.] fib(30)
 [.] fib(20)
 [.] fib(40)
 [.] fib(32)
